### 基于买卖单数据的因子

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas()
import statsmodels.api as sm

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [57]:
data23 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/data21_23.h5')

In [58]:
data23

Open  Close   High    Low  LastVolume  \
Date     InstrumentID EndTime                                           
20210104 SZ000001     09:31    19.10  18.90  19.10  18.82   3155510.0   
                      09:32    18.90  18.80  18.90  18.78   1791334.0   
                      09:33    18.80  18.84  18.85  18.81   1656900.0   
                      09:34    18.84  18.67  18.82  18.65   2376914.0   
                      09:35    18.67  18.57  18.65  18.50   2616000.0   
...                              ...    ...    ...    ...         ...   
20231012 SH689009     14:56    33.94  33.88  33.94  33.88     23275.0   
                      14:57    33.92  33.89  33.92  33.86      9297.0   
                      14:58    33.89  33.89  33.89  33.89         0.0   
                      14:59    33.89  33.89  33.89  33.89         0.0   
                      15:00    33.77  33.77  33.77  33.77     62011.0   

                               LastTurnOver  
Date     InstrumentID EndTime                
20210104 SZ000001     09:31     59855405.18  
                      09:32     33745895.32  
                      09:33     31190135.00  
                      09:34     44535547.45  
                      09:35     48631903.70  
...                                     ...  
20231012 SH689009     14:56       789311.00  
                      14:57       315093.00  
                      14:58            0.00  
                      14:59            0.00  
                      15:00      2094111.00  

[751830321 rows x 6 columns]

In [2]:
cd /mnt/datadisk2/aglv/aglv/lab_aglv/forintern/

/mnt/datadisk2/aglv/aglv/lab_aglv/forintern


In [3]:
from DataDaily import DataDaily
datadaily = DataDaily()

load ./data_daily.h5...


In [46]:
clo_day = datadaily.adjclose.loc[20210101:20231231]
ret20 = clo_day / clo_day.shift(20) - 1
ret = clo_day.pct_change()
vol = datadaily.volume.loc[20210101:20231231]

In [67]:
factor_path = '/mnt/datadisk2/aglv/aglv/aglv_factor/bs_book2/'

In [7]:
r_win = 20

buy_l_order = datadaily.buy_value_large_order.loc[20210101:20231231]
sell_l_order = datadaily.sell_value_large_order.loc[20210101:20231231]
buy_xl_order = datadaily.buy_value_exlarge_order.loc[20210101:20231231]
sell_xl_order = datadaily.sell_value_exlarge_order.loc[20210101:20231231]

buy_s_order = datadaily.buy_value_small_order.loc[20210101:20231231]
sell_s_order = datadaily.sell_value_small_order.loc[20210101:20231231]

buy_l_order_act = datadaily.buy_value_large_order_act.loc[20210101:20231231]
sell_l_order_act = datadaily.sell_value_large_order_act.loc[20210101:20231231]
buy_xl_order_act = datadaily.buy_value_exlarge_order_act.loc[20210101:20231231]
sell_xl_order_act = datadaily.sell_value_exlarge_order_act.loc[20210101:20231231]

buy_l_order_demean = buy_l_order / buy_l_order.rolling(r_win).mean()
sell_l_order_demean = sell_l_order / sell_l_order.rolling(r_win).mean()
buy_xl_order_demean = buy_xl_order / buy_xl_order.rolling(r_win).mean()
sell_xl_order_demean = sell_xl_order / sell_xl_order.rolling(r_win).mean()

buy_l_order_act_demean = buy_l_order_act / buy_l_order_act.rolling(r_win).mean()
sell_l_order_act_demean = sell_l_order_act / sell_l_order_act.rolling(r_win).mean()
buy_xl_order_act_demean = buy_xl_order_act / buy_xl_order_act.rolling(r_win).mean()
sell_xl_order_act_demean = sell_xl_order_act / sell_xl_order_act.rolling(r_win).mean()

buy_l_s_ratio = buy_l_order / buy_s_order
sell_l_s_ratio = sell_l_order / sell_s_order

In [8]:
b_s_l_ratio = buy_l_order_demean / sell_l_order_demean
b_s_l_ratio

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
date,,,,,,,,,,,,,,,,,,,,,
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# b_s_l_ratio.to_csv(factor_path + 'b_s_l_ratio.csv')

In [41]:
for i in [5, 10, 20]:
    b_r_max_std_ratio = buy_l_s_ratio / buy_l_s_ratio.rolling(i).std()
    b_r_max_std_ratio.to_csv(factor_path + f'buy_l_s_ratio_r{i}_std_ratio.csv')

In [35]:
b_b_act_l_ratio = buy_l_order / buy_l_order_act
b_b_act_l_ratio.rolling(10).mean().to_csv(factor_path + 'b_b_act_l_ratio_r10_mean.csv')

In [40]:
buy_l_s_ratio.to_csv(factor_path + 'buy_l_s_ratio.csv')

In [12]:
buy_l_order_ret20_ratio = buy_l_order / ret20
buy_l_order_ret20_ratio.to_csv(factor_path + 'buy_l_order_ret20_ratio.csv')

In [13]:
buy_l_order_ret20_ratio_r10std_ratio = buy_l_order_ret20_ratio / buy_l_order_ret20_ratio.rolling(10).std()
buy_l_order_ret20_ratio_r10std_ratio.to_csv(factor_path + 'buy_l_order_ret20_ratio_r10sd_ratio.csv')

In [18]:
buy_l_order_corr_ret = buy_l_order_demean.rolling(20).corr(ret)
buy_l_order_corr_ret.to_csv(factor_path + 'buy_l_order_corr_ret.csv')

In [29]:
def neutralize_stock(factor_stock):
    factor_stock = factor_stock.fillna(0)
    stock_name = factor_stock.name
    # X = pd.DataFrame({'ret20':ret20.loc[factor_stock.index, stock_name], 'vol20':vol20.loc[factor_stock.index, stock_name]})
    X = ret20.loc[factor_stock.index, stock_name]
    X = X.fillna(0)
    X = sm.add_constant(X)
    model = sm.OLS(factor_stock, X)
    res = model.fit()
    return res.resid

In [39]:
# neu_buy_l_order_demean = buy_l_order_demean.loc[ret20.index, ret20.columns].rolling(20).progress_apply(neutralize_stock)
# neu_buy_l_order_demean

In [43]:
buy_l_order_act_demean.rolling(20).corr(ret).mean(axis=1)

date
20210104         NaN
20210105         NaN
20210106         NaN
20210107         NaN
20210108         NaN
              ...   
20231225    0.327817
20231226    0.309683
20231227    0.306316
20231228    0.370856
20231229    0.397569
Length: 727, dtype: float64

In [44]:
79 / 218

0.3623853211009174

In [47]:
exce_buy_l_order = buy_l_order - buy_l_order.shift(1) * (vol / vol.shift(1))

In [48]:
exce_buy_l_order.to_csv(factor_path + 'exce_buy_l_order.csv')

In [49]:
exce_buy_l_order_r10_std_ratio = exce_buy_l_order / exce_buy_l_order.rolling(10).std()
exce_buy_l_order_r10_std_ratio.to_csv(factor_path + 'exce_buy_l_order_r10_std_ratio.csv')

In [50]:
exce_buy_l_order_demean = exce_buy_l_order / exce_buy_l_order.rolling(20).mean()

In [51]:
exce_buy_l_order_demean.to_csv(factor_path + 'exce_buy_l_order_demean.csv')

In [52]:
exce_buy_l_order_demean_r10_std_ratio = exce_buy_l_order_demean / exce_buy_l_order_demean.rolling(10).std()
exce_buy_l_order_demean_r10_std_ratio.to_csv(factor_path + 'exce_buy_l_order_demean_r10_std_ratio.csv')

In [53]:
exce_buy_l_order_per = exce_buy_l_order / vol
exce_buy_l_order_per.to_csv(factor_path + 'exce_buy_l_order_per.csv')

In [54]:
exce_buy_s_order = buy_s_order - buy_s_order.shift(1) * (vol / vol.shift(1))

In [55]:
exce_buy_s_order.to_csv(factor_path + 'exce_buy_s_order.csv')

In [60]:
exce_ret_buy_l_order = buy_l_order - buy_l_order.shift(1) * (clo_day / clo_day.shift(1))

In [62]:
exce_ret_buy_l_order_demean = exce_ret_buy_l_order / exce_ret_buy_l_order.rolling(10).mean()
exce_ret_buy_l_order.to_csv(factor_path + 'exce_ret_buy_l_order_demean.csv')

In [76]:
exce_buy_l_order_demean.mean(axis=1).isna().sum()

20

In [87]:
exce_ret_buy_l_order_demean

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
date,,,,,,,,,,,,,,,,,,,,,
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
def _f(x:pd.Series):
    return np.power((x - x.mean()), 2)

In [93]:
cd_exce_ret_buy_l_order_demean = exce_ret_buy_l_order_demean.apply(_f, axis=1)

In [82]:
exce_ret_buy_l_order_demean

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
date,,,,,,,,,,,,,,,,,,,,,
20210104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20231227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
cd_exce_ret_buy_l_order_demean.to_csv(factor_path + 'cd_exce_ret_buy_l_order_demean.csv')

In [94]:
cd_exce_ret_buy_l_order_demean.isna().sum(axis=1)

date
20210104    5644
20210105    5644
20210106    5644
20210107    5644
20210108    5644
            ... 
20231225     578
20231226     580
20231227     579
20231228     581
20231229     581
Length: 727, dtype: int64

In [119]:
buy_l_order_act_demean.to_csv(factor_path + 'buy_l_order_act_demean.csv')

In [120]:
neu_ret_buy_l_order_act_demean = buy_l_order_act_demean - buy_l_order_act_demean.shift(1) * (clo_day / clo_day.shift(1))

In [121]:
neu_ret_buy_l_order_act_demean.to_csv(factor_path + 'neu_ret_buy_l_order_act_demean.csv')

In [ ]:
,